<a href="https://colab.research.google.com/github/tonyjung01/Machine-Learning/blob/main/5-3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 트리의 앙상블

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/rickiepark/hg-mldl/blob/master/5-3.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />구글 코랩에서 실행하기</a>
  </td>
</table>

데이터는 정형 데이터와 비정형 데이터가 있음  
정형은 대부분 다르는 왠만한 애들 엑셀 등에 저장되는 애들  
비정형은 텍스트 사진 등등...  
정형 데이터를 다루는 데에 가장 뛰어난 성과를 내는 알고리즘 앙상블 학습(대부분 결정 트리 기반)  
비정형은 신경망 사용

## 랜덤포레스트

결정 트리를 랜덤하게 만들어 숲을 만들고 (tree로 forest) 각 결정 트리의 예측을 사용해 최종 예측 만듬

부트스트랩 샘플 (복원 추출로 만듬)을 사용  
노드 분할할 때 전체 특성 중 무작위로 일부 특성 고르고 이 중 최선의 분할 찾음  
  
  분류 모델인 RandomForestClassifier는 기본적으로 전체 특성 개수의 제곱근 개 만큼 선택  
  회귀 모델인 RandomForestRegressor는 전체 특성 사용

사이킷런의 랜덤 포레스트는 기본적으로 100개의 결정 트리를 이런 방식으로 훈련함  
분류일 때는 각 트리의 클래스 확률 평균하여 가장 높은 확률 가진 클래스 예측으로 삼음  
회귀일 때는 단순히 각 트리의 예측 평균

랜덤 포레스트는 랜덤으로 선택한 샘플과 특성 사용하므로 과대적합 방지되고 검증 세트 테스트 세트에서 안정적인 성능 얻을 수 있음

In [4]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

wine = pd.read_csv('https://bit.ly/wine_csv_data')

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [5]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


100개의 결정 트리 사용하므로 RandomForestClassifier의 n_jobs -1로 하여 모든 CPU 코어 사용  
cross_validate의 n_jobs도 -1로 하여 최대한 병렬로 교차검증 수행  
return_train_score True로 하여 검증 점수와 훈련 세트 점수도 함께 반환 (디폴트는 false)  
다소 과대적합이긴한데 알고리즘 조사 목적이므로 일단 조정안함

In [6]:
rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


각 특성의 중요도 확인  
앞의 결정 트리에서보다 더 균등하게 나옴 -> 특성의 일부를 랜덤하게 선택하여 훈련하기 때문에 하나에 과도하게 집중하지 않고 더 많은 특성이 훈련에 기여하게 도움  
과대적합 줄이고 일반화 성능 향상

In [7]:
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)

rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


복원 추출을 원래 표본수만큼 하기 때문에 아예 사용되지 않는 샘플 생김 (OOB 샘플)  
이를 이용하여 마치 검증 세트처럼 사용 가능  
oob_score true로 해야함(디폴트 false)  
교차 검증에서와 비슷한 점수 얻음  
얘 쓰면 교차 검증 대신할 수 있어서 더 많은 샘플 사용 가능

## 엑스트라트리

기본적인 매커니즘은 랜덤 포레스트와 비슷하지만 부트스트랩 샘플 사용X  
결정 트리를 만들 때 전체 훈련 세트 사용  
대신 노드 분할할 때 가장 좋은 분할을 찾지 않고 무작위로 분할함

하나의 결정 트리에서 특성 무작위 분할하면 성능 낮아지지만 많은 트리 앙상블하므로 오히려 과대적합 막고 검증 세트 점수 높임

In [8]:
from sklearn.ensemble import ExtraTreesClassifier

et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


랜덤 포레스트와 비슷  
엑스트라 트리가 무작위성이 더 크므로 랜덤 포레스트보다 더 많은 트리 훈련해야함(엑스트라 트리는 개별 트리의 예측력이 상대적으로 낮기 때문에, 더 많은 트리를 사용하면 성능이 안정적으로 수렴할 가능성이 높음)    
랜덤하게 노드 분할하므로 계산 속도 빠름

In [9]:
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


## 그레이디언트 부스팅

깊이가 얕은 결정 트리를 사용하여 이전 트리의 오차 보완  
기본적으로 깊이가 3인 결정 트리 100개 -> 과대적합 방지, 높은 일반화 성능

In [10]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


과대적합x 결정트리 개수 늘려도 과대적합에 매우 강함

In [11]:
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


결정트리 수를 500으로 늘렸지만 과대적합 잘 억제함

In [12]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.15872278 0.68010884 0.16116839]


랜덤 포레스트보다 일부 특성(여기서는 당도)에 더 집중함

트리 훈련에 사용할 훈련 세트의 비율을 정하는 매개변수 subsample  
기보은 1.0으로 모두 이용하는데 확률적 경사 하강법이나 미니배치 경사 하강법처럼 일부 샘플 랜덤하게 선택하여 학습 진행 가능

일반적으로 그레이디언트 부스팅 > 랜덤 포레스트  
but 순서대로 트리 추가하므로 훈련 속도 느림

## 히스토그램 기반 부스팅

정형 데이터를 다루는 머신러닝 알고리즘 중 가장 인기가 많음  
입력 특성을 256개 구간으로 나눔 -> 노드를 분할할 때 최적의 분할을 매우 빠르게 찾을 수 있음  
이 구간 중 하나를 떼어 놓고 누락된 값을 위해 사용하므로 따로 전처리가 필요없음

In [13]:
# 사이킷런 1.0 버전 아래에서는 다음 라인의 주석을 해제하고 실행하세요.
# from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9321723946453317 0.8801241948619236


과대적합을 잘 억제하면서 그레이디언트 부스팅보다 높은 성능 보여줌

In [14]:
from sklearn.inspection import permutation_importance

hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target, n_repeats=10,
                                random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]


In [15]:
result = permutation_importance(hgb, test_input, test_target, n_repeats=10,
                                random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


왜째서 특성 중요도값의 합이 1이 아닌가?  
✔ 랜덤 포레스트와 XGBoost는 특성 중요도를 정규화하여 합이 1이 되도록 하지만, HGB는 그렇지 않음.  
✔ HGB는 특성이 기여한 절대적인 중요도를 직접 출력하므로 합이 1이 아닐 수 있음.  
✔ HGB는 정규화 효과가 강하여 특성 중요도가 분산되거나 특정 특성이 낮은 중요도를 가질 가능성이 높음.  
✔ 이것이 모델이 잘못 학습되었다는 의미는 아니므로, HGB에서 중요도 값을 직접 비교하는 것이 더 적절함.  

In [16]:
hgb.score(test_input, test_target)

0.8723076923076923

앙상블 모델이 단일 결정 트리보다 좋다(랜덤 서치는 86%)  
근데 이게 1.2%p차인데 더 좋다고 할 수 있는거임..??

#### XGBoost

사이킷런 말고도 그레이디언트 부스팅 알고리즘 구현한 라이브러리가 여럿 있음  
XGBoost가 대표적임

In [17]:
!pip install scikit-learn==1.3.1


In [18]:
!pip install xgboost --upgrade

In [19]:
from xgboost import XGBClassifier

xgb = XGBClassifier(tree_method='hist', random_state=42)
scores = cross_validate(xgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9558403027491312 0.8782000074035686


XGBoost가 오류가 나서 사이킷런 낮추고 xgboost 업그레이드 하고 해봄 어떤게 먹힌건지는 모르겠는데 일단 됨

#### LightGBM

In [20]:
from lightgbm import LGBMClassifier

lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

/usr/local/lib/python3.11/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


0.935828414851749 0.8801251203079884


앙상블 학습 알고리즘
*   랜덤 포레스트 : 가장 대표적, 부트스트랩 샘플, 전체 특성 중 일부 랜덤 선택
*   엑스트라 트리 : 랜덤 포레스트와 비슷하지만 부트스트랩x, 훈련 속도 빠르지만 트리 더 많이 필요
*   그레이디언트 부스팅 : 성능 굿, 훈련속도 느림(병렬x) 매개변수 크면 과대적합 위험
*   히스토그램 기반 그.부. : 훈련 데이터를 256개 구간으로 변환하므로 노드 분할 속도 빠름



